# <strong><u>1: Video Processing Notebook |</u></strong>

This notebook is an implementation of video processing using MediaPipe, Google's pose estimation model. The process extracts x, y movement of the body using markerless motion capture for ## articulators, or keypoints. 

In this workflow, we output the x,y movement for the keypoints in the upper body that are particularly relevant for co-speech gesture research: `right_shoulder`, `left_shoulder`, `right_elbow`, `left_elbow`,  `right_wrist`,  `left_wrist`, `right_eye`,  `left_eye`, `nose`. It is possible to track other keypoints, as is documented in more detail in the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/pose_landmarker). For this workflow, later stages of processing will further subset these keypoints for a specific analysis.

This script takes a video input file, uses the MediaPipe model to track body movement, and outputs a dataframe with the time scale and x, y coordinates for the upperbody articulators specified above. The dataframe is then written to a csv file.

This script assumes that you have cloned the Co-Speech-Gesture-Automation github repository and that your input files are stored in the VIDEO_FILES folder. Your outputs will be written to the MOTION_TRACKING_FILES folder.

### <strong>Requirements</strong>

To run this notebook, you will need the following Python packages:

- mediapipe
- opencv-python
- pandas
- matplotlib

You can install these packages using pip:
```shell
    pip install mediapipe opencv-python pandas matplotlib
```



### <strong>Importing Libraries</strong>
Here, we import essential libraries:
- `pandas`: For DataFrame manipulation and data analysis
- `numpy`: For numerical computations
- `plotly`: For data visualization

In [2]:
import cv2
import mediapipe as mp
import os
import pandas as pd
import tqdm as tq

### <strong>Setting Parameters</strong>
In this section, you can modify the following parameters:
- `MODEL`: Choose between Lite model (`1`) and Full model (`2`). Lite Model (`1`) is the  `Default`. 

The lite model is faster and relatively accurate, but using the full model will improve tracking accuracy.

- `video_path`: Path to the video file.


This cell also provides you a way to check that the file and directory specified in `video_path` can be found before beginning video processing. If the file path was found, you can proceed to the video processing initialization and video processing steps. If your file was not found and you should amend your `video_path` before proceeding.

In [3]:
MODEL = 2  # 1 = Lite model, 2 = Full model
video_path = "../Videos/Test_Video.MOV"
video_extensions = ["*.mp4", "*.avi", "*.MOV"]

if os.path.exists(video_path) == True:
    print(f"{video_path} is a valid file. Proceed with processing.")
else:
    raise ValueError(f"{video_path} does not exist. Try adding the entire file path.")

../Videos/Test_Video.MOV is a valid file. Proceed with processing.


### <strong>Initialization</strong>
This part initializes MediaPipe components used in the notebook.

There is no need to do anything with this cell before running it.

In [4]:
# Initialize MediaPipe components
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

### <strong>Keypoint Tracking</strong>
The core logic of video processing is performed in this loop.
This code block reads in a video file and extracts pose landmarks from each frame of the video using the `mediapipe` and `opencv-python` libraries. The pose landmarks are then stored in a DataFrame along with the corresponding time stamp.

This cell can be run without making any changes unless you require tracking for keypoints not specified in this workflow.


In [5]:
data = []
with mp_holistic.Holistic(static_image_mode=False, model_complexity=MODEL, 
                          min_detection_confidence=0.4, min_tracking_confidence=0.4) as holistic:
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Get the frame width and height from the video capture object
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Define the codec using VideoWriter_fourcc and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output.avi', fourcc, fps, (frame_width, frame_height), isColor=True)
    
    for _ in tq.tqdm(range(total_frames), desc="Processing frames", unit="frame"):
        ret, image = cap.read()
        if not ret:
            break
        
        frame_number = cap.get(cv2.CAP_PROP_POS_FRAMES)
        time_ms = round((frame_number / fps) * 1000, 0)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)
        
        # Convert the image color back to BGR for VideoWriter
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw the pose landmarks on the image
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        
        # Write the frame into the file 'output.avi'
        out.write(image)
        
        if results.pose_landmarks is not None:
            right_shoulder_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x
            right_shoulder_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y
            left_shoulder_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].x
            left_shoulder_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y
            right_elbow_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].x
            right_elbow_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y
            left_elbow_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].x
            left_elbow_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y
            right_wrist_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x
            right_wrist_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y
            left_wrist_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].x
            left_wrist_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y
            right_eye_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].x
            right_eye_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].y
            left_eye_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].x
            left_eye_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].y
            nose_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x
            nose_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y
            right_index_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].x
            right_index_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].y

            data.append([time_ms, right_shoulder_x, right_shoulder_y, left_shoulder_x, left_shoulder_y, 
                        right_elbow_x, right_elbow_y, left_elbow_x, left_elbow_y, 
                        right_wrist_x, right_wrist_y, left_wrist_x, left_wrist_y, 
                        right_eye_x, right_eye_y, left_eye_x, left_eye_y, right_index_x, right_index_y,
                        nose_x, nose_y])
            
                    
        # out.write(image)

    cap.release()
    # out.release()

Processing frames:   0%|          | 0/2532 [00:00<?, ?frame/s]C:\Users\cosmo\AppData\Roaming\Python\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
Processing frames:  54%|█████▍    | 1375/2532 [01:29<01:15, 15.40frame/s]

### <strong>Converting the data to a dataframe</strong>
Here the keypoint data is converted to a dataframe to preview before writing to a csv file. All keypoint data are saved at this stage of processing, and there is no need to make changes to the articulators listed here; however, later stages of processing will subset the data to a chosen articulator.

In [ ]:
# Dictionary of all keypoints, appended to final dataframe
df = pd.DataFrame(data, columns=[
    "time_ms", 
    "right_shoulder_x", "right_shoulder_y", 
    "left_shoulder_x", "left_shoulder_y", 
    "right_elbow_x", "right_elbow_y", 
    "left_elbow_x", "left_elbow_y", 
    "right_wrist_x", "right_wrist_y", 
    "left_wrist_x", "left_wrist_y", 
    "right_eye_x", "right_eye_y", 
    "left_eye_x", "left_eye_y",
    "nose_x", "nose_y",
    "right_index_x", "right_index_y" 
    ])

### <strong>Data Output</strong>
Finally, the processed data is stored in a DataFrame and saved as a pickle file or csv file. For testing we recommend using a csv file and then moving to a pickle file for processing larger datasets because pickle files are more efficient for processing and storing.

In [ ]:
# Save DataFrame as CSV file
csv_file_name = "../Keypoints/" + os.path.splitext(os.path.basename(video_path))[0] + "_keypoints.csv"
df.to_csv(csv_file_name, index=False)
print(f"DataFrame saved as {csv_file_name}")